In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Airai,TL,2020-10-30 05:23:31,-8.93,125.41,77.85,55,8,4.07
1,1,Punta Arenas,CL,2020-10-30 05:23:31,-53.15,-70.92,46.40,70,75,23.04
2,2,Rudiskes,LT,2020-10-30 05:23:31,54.52,24.83,46.99,100,90,6.93
3,3,Hithadhoo,MV,2020-10-30 05:23:31,-0.60,73.08,83.32,76,81,12.53
4,4,Jamestown,US,2020-10-30 05:23:32,42.10,-79.24,42.01,93,90,13.87


In [3]:
# Get the data types.
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig 


Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Airai,TL,2020-10-30 05:23:31,-8.93,125.41,77.85,55,8,4.07
3,3,Hithadhoo,MV,2020-10-30 05:23:31,-0.60,73.08,83.32,76,81,12.53
9,9,Hilo,US,2020-10-30 05:23:33,19.73,-155.09,77.00,88,90,3.27
10,10,Rikitea,PF,2020-10-30 05:23:06,-23.12,-134.97,75.45,73,100,4.23
12,12,Olinda,BR,2020-10-30 05:23:35,-8.01,-34.86,80.60,74,40,8.05
13,13,Souillac,MU,2020-10-30 05:23:35,-20.52,57.52,75.00,51,16,11.99
16,16,Palauig,PH,2020-10-30 05:23:38,15.44,119.90,84.78,71,19,5.64
17,17,Victoria,HK,2020-10-30 05:20:04,22.29,114.16,80.01,65,40,6.93
21,21,Vila Velha,BR,2020-10-30 05:23:39,-20.33,-40.29,77.00,88,75,6.93
25,25,Saint-Philippe,RE,2020-10-30 05:23:41,-21.36,55.77,75.20,53,75,23.04


In [9]:
preferred_cities_df.count()

City_ID       169
City          169
Country       169
Date          169
Lat           169
Lng           169
Max Temp      169
Humidity      169
Cloudiness    169
Wind Speed    169
dtype: int64

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Airai,TL,77.85,-8.93,125.41,
3,Hithadhoo,MV,83.32,-0.60,73.08,
9,Hilo,US,77.00,19.73,-155.09,
10,Rikitea,PF,75.45,-23.12,-134.97,
12,Olinda,BR,80.60,-8.01,-34.86,
13,Souillac,MU,75.00,-20.52,57.52,
16,Palauig,PH,84.78,15.44,119.90,
17,Victoria,HK,80.01,22.29,114.16,
21,Vila Velha,BR,77.00,-20.33,-40.29,
25,Saint-Philippe,RE,75.20,-21.36,55.77,


In [11]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [12]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
     print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Airai,TL,77.85,-8.93,125.41,Juvinal Martin
3,Hithadhoo,MV,83.32,-0.60,73.08,Scoop Guest House
9,Hilo,US,77.00,19.73,-155.09,Hilo Hawaiian Hotel
10,Rikitea,PF,75.45,-23.12,-134.97,Pension Maro'i
12,Olinda,BR,80.60,-8.01,-34.86,Hotel 5 Sóis
...,...,...,...,...,...,...
565,Road Town,VG,82.99,18.42,-64.62,"ZINGARA 76ft Catamaran, full crew-all included"
566,Oussouye,SN,77.92,12.48,-16.55,Auberge du Routard
569,Bahia Honda,CU,77.54,22.90,-83.16,Casa Gualberto - Deysi
570,Maues,BR,79.14,-3.38,-57.72,Hotel Miramar


In [14]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [20]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))